
## CNN With Custom Images

The dataset contains 2 folders - Infected - Uninfected

And a total of 27,558 images.


This Dataset is taken from the official NIH Website: https://ceb.nlm.nih.gov/repositories/malaria-datasets/ 

In [9]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# my_data_dir = '/content/drive/MyDrive/Clarusway - ML2/3- Classes & Labs/2- DL/DL-S7-(CNN_Image_Classification_Malaria)_12_Mar_2022/cell_images'

In [12]:
my_data_dir = '../input/dog-cat/data'

In [ ]:
# pwd

In [13]:
# CONFIRM THAT THIS REPORTS BACK 'test', and 'train'
os.listdir(my_data_dir) 

In [16]:
test_path = my_data_dir+'/test/'
train_path = my_data_dir+'/train/'
validation_pat = my_data_dir+'/validation/'

In [19]:
os.listdir(test_path)

In [20]:
os.listdir(train_path)

In [22]:
os.listdir(train_path+'cat')

In [25]:
os.listdir(train_path+'cat')[7]

In [28]:
cat_img = train_path+'cat'+'/cat.2842.jpg'

In [26]:
imread(cat_img)

In [30]:
cat_img = imread(cat_img)

In [31]:
plt.imshow(cat_img)

In [32]:
cat_img.shape

In [33]:
os.listdir(train_path+'dog')

In [34]:
dog_path = train_path+'dog/'+os.listdir(train_path+'dog')[7]

In [35]:
imread(dog_path)

In [36]:
dog_img = imread(dog_path)
plt.imshow(dog_img)

**Let's check how many images there are.**

In [37]:
len(os.listdir(train_path+'cat'))

In [38]:
len(os.listdir(train_path+'dog'))

**Let's find out the average dimensions of these images.**

In [39]:
dog_img.shape

In [40]:
cat_img.shape

In [47]:
x= []
y = []
for image in os.listdir(train_path):
    
    img = imread(train_path+image)
    d1,d2,colors = img.shape
    x.append(d1)
    y.append(d2)

In [43]:
y

In [ ]:
x

In [44]:
sns.scatterplot(x,y)

In [45]:
np.mean(x)

In [46]:
np.mean(y)

scale-up ve scale-down yapmam gerekecek. bana en uygun boyutu belirledim.

In [ ]:
image_shape = (130,130,3)

## Preparing the Data for the model

In [ ]:
uninfected_cell.max()

In [ ]:
para_img.max()

Scale etmeme gerek yok. Cunku 0-1 arasinda degerler cikti.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

resimleri scale ederken 'smart_resize' kullaniyor. >>help oku.

In [ ]:
help(ImageDataGenerator)

In [ ]:
image_gen = ImageDataGenerator(rotation_range=15, 
                               width_shift_range=0.10, 
                               height_shift_range=0.10, 
                               #rescale=1/255, 
                               shear_range=0.1, 
                               zoom_range=0.1, 
                               horizontal_flip=True, 
                               fill_mode='nearest')

Bu fonksiyon rasgele resimleri bozuyor. Hicbir zaman gercek resmi modele gostermiyor. Her bir resme bozma islemi uyguluyor.

Bozma ozelliklerini parametrelerle belirledik. Her resme farkli bir bozma uygulaniyor. Rasgele karar veriyor.

resimlerimiz scale olmus halde oldugu icin rescale kapattik.

rotation en fazla 15 derece olsun dedik.

kaydirma (shift) orani en fazla %10 
zoom in/out en falza %10, ama rasgele yapiyor.

In [ ]:
plt.imshow(para_img)

In [ ]:
plt.imshow(image_gen.random_transform(para_img))

In [ ]:
image_gen.flow_from_directory(train_path)

In [ ]:
image_gen.flow_from_directory(test_path)

Resimlerimi okudu. Ama kopyalamadi. Yer yetmez. Her batch icin sirasi geldiginde okuyacak.:

# Creating the Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D

In [ ]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())


model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.summary()

## Early Stopping

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
early_stop = EarlyStopping(monitor='val_loss',patience=5)

## Training the Model

In [ ]:
#help(image_gen.flow_from_directory)

In [ ]:
batch_size = 16
train_image_gen = image_gen.flow_from_directory(train_path,
                                               target_size=image_shape[:2],
                                                color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='binary')

In [ ]:
test_image_gen = image_gen.flow_from_directory(test_path,
                                               target_size=image_shape[:2],
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='binary')

In [ ]:
train_image_gen.class_indices

In [ ]:
test_image_gen.class_indices

In [ ]:
results = model.fit_generator(train_image_gen,epochs=8,
                              validation_data=test_image_gen, callbacks=[early_stop])

# Evaluating the Model

In [ ]:
summary = pd.DataFrame(model.history.history)
summary.head()

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(summary.loss, label="loss")
plt.plot(summary.val_loss, label="val_loss")
plt.legend(loc="upper right")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(summary.accuracy, label="accuracy")
plt.plot(summary.val_accuracy, label="val_accuracy")
plt.legend(loc="upper left")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.show()

In [ ]:
model.metrics_names

In [ ]:
model.evaluate_generator(test_image_gen)

In [ ]:
from tensorflow.keras.preprocessing import image

In [ ]:
pred_probabilities = model.predict_generator(test_image_gen)

In [ ]:
pred_probabilities

In [ ]:
test_image_gen.classes

In [ ]:
predictions = pred_probabilities > 0.5

In [ ]:
predictions

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
print(classification_report(test_image_gen.classes,predictions))

In [ ]:
confusion_matrix(test_image_gen.classes,predictions)

In [ ]:
model.save('malaria_model.h5')

# Predicting on an Image

In [ ]:
para_cell

In [ ]:
my_image = image.load_img(para_cell,target_size=image_shape)

In [ ]:
my_image

In [ ]:
type(my_image)

In [ ]:
my_image = image.img_to_array(my_image)

In [ ]:
type(my_image)

In [ ]:
my_image.shape

In [ ]:
my_image = np.expand_dims(my_image, axis=0)

In [ ]:
my_image.shape

In [ ]:
model.predict(my_image)

In [ ]:
train_image_gen.class_indices

In [ ]:
test_image_gen.class_indices